In [1]:
import os

In [2]:
# + SubVert

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

In [4]:
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
 
config = tf.ConfigProto()  
config.gpu_options.allow_growth=True  
session = tf.Session(config=config)
 
KTF.set_session(session)

/home/qit16/anaconda3/envs/qt_torch/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/qit16/anaconda3/envs/qt_torch/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/qit16/anaconda3/envs/qt_torch/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/qit16/anaconda3/envs/qt_torch/lib/pyt

In [5]:
import csv
import datetime
import time
import json
import itertools
import random
import os
from sklearn.metrics import roc_auc_score

In [6]:
import nltk 
from nltk.tokenize import word_tokenize
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_fscore_support

In [7]:
import numpy
import keras
from keras.utils.np_utils import *
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding, concatenate
from keras.layers import Dense, Input, Flatten, average,Lambda

from keras.layers import *
from keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from keras.utils import plot_model
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from keras.optimizers import *

In [8]:
import os

In [9]:
from datetime import datetime
import time
def trans2tsp(timestr):
    return int(time.mktime(datetime.strptime(timestr, '%m/%d/%Y %I:%M:%S %p').timetuple()))

In [10]:
def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

In [11]:
def shuffle(pn,labeler,pos):
    index=np.arange(pn.shape[0])
    pn=pn[index]
    labeler=labeler[index]
    pos=pos[index]
    
    for i in range(pn.shape[0]):
        index=np.arange(npratio+1)
        pn[i,:]=pn[i,index]
        labeler[i,:]=labeler[i,index]
    return pn,labeler,pos

In [12]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [13]:
#REad News

In [14]:
def read_news(path,filenames):
    news={}
    category=[]
    subcategory=[]
    news_index={}
    index=1
    word_dict={}
    word_index=1
    with open(os.path.join(path,filenames)) as f:
        lines=f.readlines()
    for line in lines:
        splited = line.strip('\n').split('\t')
        doc_id,vert,subvert,title= splited[0:4]
        news_index[doc_id]=index
        index+=1
        category.append(vert)
        subcategory.append(subvert)
        title = title.lower()
        title=word_tokenize(title)
        news[doc_id]=[vert,subvert,title]
        for word in title:
            word = word.lower()
            if not(word in word_dict):
                word_dict[word]=word_index
                word_index+=1
    category=list(set(category))
    subcategory=list(set(subcategory))
    category_dict={}
    index=1
    for c in category:
        category_dict[c]=index
        index+=1
    subcategory_dict={}
    index=1
    for c in subcategory:
        subcategory_dict[c]=index
        index+=1
    return news,news_index,category_dict,subcategory_dict,word_dict

In [15]:
data_root_path = '/data1/qitao/MIND-Full'
embedding_path = '/data/data/qit/'

In [16]:
%time news,news_index,category_dict,subcategory_dict,word_dict = read_news(data_root_path,'docs.tsv')

CPU times: user 31.7 s, sys: 1.54 s, total: 33.3 s
Wall time: 33.3 s


In [17]:
MAX_SENTENCE = 30

In [18]:
def get_doc_input(news,news_index,category,subcategory,word_dict):
    news_num=len(news)+1
    news_title=np.zeros((news_num,MAX_SENTENCE),dtype='int32')
    news_vert=np.zeros((news_num,),dtype='int32')
    news_subvert=np.zeros((news_num,),dtype='int32')
    for key in news:    
        vert,subvert,title=news[key]
        doc_index=news_index[key]
        news_vert[doc_index]=category[vert]
        news_subvert[doc_index]=subcategory[subvert]
        for word_id in range(min(MAX_SENTENCE,len(title))):
            news_title[doc_index,word_id]=word_dict[title[word_id].lower()]
        
    return news_title,news_vert,news_subvert

In [19]:
news_title,news_vert,news_subvert=get_doc_input(news,news_index,category_dict,subcategory_dict,word_dict)

In [20]:
def load_matrix(embedding_path,word_dict):
    embedding_matrix = np.zeros((len(word_dict)+1,300))
    have_word=[]
    with open(os.path.join(embedding_path,'glove.840B.300d.txt'),'rb') as f:
        while True:
            l=f.readline()
            if len(l)==0:
                break
            l=l.split()
            word = l[0].decode()
            if word in word_dict:
                index = word_dict[word]
                tp = [float(x) for x in l[1:]]
                embedding_matrix[index]=np.array(tp)
                have_word.append(word)
    return embedding_matrix,have_word

In [21]:
title_word_embedding_matrix, have_word = load_matrix(embedding_path,word_dict)

In [22]:
#End Read News

In [23]:
# Read Session

In [24]:
def read_clickhistory(path,filename):
    
    lines = []
    userids = []
    with open(os.path.join(data_root_path,filename)) as f:
        lines = f.readlines()
        
    sessions = []
    for i in range(len(lines)):
        _,click, imp = lines[i].strip().split('\t')
        clikcs = click.split('#N#')
        true_click = []
        for click in clikcs:
            t = click.split('#TAB#')[0]
            if t =='':
                continue
            true_click.append(t)
        pos, neg, _ = imp.split('#TAB#')
        pos = pos.split()
        neg = neg.split()
        sessions.append([true_click,pos,neg])
    return sessions

In [25]:
train_session = read_clickhistory(data_root_path,'train_sam2.tsv')

In [26]:
test_session = read_clickhistory(data_root_path,'test.tsv')

In [27]:
# val_session = read_clickhistory(data_root_path,'val_sam2.tsv')

In [28]:
#Parser User

In [29]:
MAX_ALL = 50

In [30]:
def parse_user(session):
    user_num = len(session)
    user={'click': np.zeros((user_num,MAX_ALL),dtype='int32'),}
    for user_id in range(len(session)):
        tclick = []
        click, pos, neg =session[user_id]
        for i in range(len(click)):
            tclick.append(news_index[click[i]])
        click = tclick

        if len(click) >MAX_ALL:
            click = click[-MAX_ALL:]
        else:
            click=[0]*(MAX_ALL-len(click)) + click
            
        user['click'][user_id] = np.array(click)
    return user

In [31]:
train_user = parse_user(train_session)

In [32]:
test_user = parse_user(test_session)

In [33]:
# val_user = parse_user(val_session)

In [34]:
#Get Training Data

In [35]:
npratio=1

In [36]:
def get_train_input(session):
    sess_pos = []
    sess_neg = []
    user_id = []
    for sess_id in range(len(session)):
        sess = session[sess_id]
        _, poss, negs=sess
        for i in range(len(poss)):
            pos = poss[i]
            neg=newsample(negs,npratio)
            sess_pos.append(pos)
            sess_neg.append(neg)
            user_id.append(sess_id)
    print(len(user_id))
    sess_all = np.zeros((len(sess_pos),1+npratio),dtype='int32')
    label = np.zeros((len(sess_pos),1+npratio))
    for sess_id in range(sess_all.shape[0]):
        pos = sess_pos[sess_id]
        negs = sess_neg[sess_id]
        sess_all[sess_id,0] = news_index[pos]
        index = 1
        for neg in negs:
            sess_all[sess_id,index] = news_index[neg]
            index+=1
        #index = np.random.randint(1+npratio)
        label[sess_id,0]=1
    user_id = np.array(user_id, dtype='int32')
    
    return sess_all, user_id, label

In [37]:
train_sess, train_user_id, train_label = get_train_input(train_session)

3383654


In [38]:
def get_test_input(session):
    
    Impressions = []
    userid = []
    for sess_id in range(len(session)):
        _, poss, negs = session[sess_id]
        imp = {'labels':[],
                'docs':[]}
        userid.append(sess_id)
        for i in range(len(poss)):
            docid = news_index[poss[i]]
            imp['docs'].append(docid)
            imp['labels'].append(1)
        for i in range(len(negs)):
            docid = news_index[negs[i]]
            imp['docs'].append(docid)
            imp['labels'].append(0)
        Impressions.append(imp)
        
    userid = np.array(userid,dtype='int32')
    
    return Impressions, userid,

In [39]:
test_impressions, test_userids = get_test_input(test_session)

In [40]:
# val_impressions, val_userids = get_test_input(val_session)

In [70]:
from keras.utils import Sequence

class get_hir_train_generator(Sequence):
    def __init__(self,news_scoring,clicked_news,user_id, news_id, label, batch_size):
        self.news_emb = news_scoring
        self.clicked_news = clicked_news

        self.user_id = user_id
        self.doc_id = news_id
        self.label = label
        
        self.batch_size = batch_size
        self.ImpNum = self.label.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.ImpNum / float(self.batch_size)))
    
    def __get_news(self,docids):
        news_emb = self.news_emb[docids]

        return news_emb
        

    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed> self.ImpNum:
            ed = self.ImpNum
        label = self.label[start:ed].argmax(axis=-1)
        
        doc_ids = self.doc_id[start:ed]
        title= self.__get_news(doc_ids)
        
        user_ids = self.user_id[start:ed]
        clicked_ids = self.clicked_news[user_ids]
        user_title = self.__get_news(clicked_ids)
        
        click_mask = clicked_ids>0
        click_mask = np.array(click_mask,dtype='float32')
        
        click_num = click_mask.sum(axis=-1)
        click_num = np.array(click_num,dtype='int32')
        click_num = click_num.reshape((len(click_num),1))
        
        label = np.array(label,dtype='int64')
        label = torch.LongTensor(label).cuda()
        click_num = torch.LongTensor(click_num).cuda()
        click_mask = torch.FloatTensor(click_mask).cuda()
        title = torch.LongTensor(title).cuda()
        user_title = torch.LongTensor(user_title).cuda()



        return (title, user_title,click_mask,click_num,label)

In [71]:
train_generator = get_hir_train_generator(news_title,train_user['click'],train_user_id,train_sess,train_label,32)

In [72]:
class get_hir_user_generator(Sequence):
    def __init__(self,news_emb,clicked_news,batch_size):
        self.news_emb = news_emb

        self.clicked_news = clicked_news

        self.batch_size = batch_size
        self.ImpNum = self.clicked_news.shape[0]
        
    def __len__(self):
        return int(np.ceil(self.ImpNum / float(self.batch_size)))

    
    def __get_news(self,docids):
        news_emb = self.news_emb[docids]

        return news_emb
    
    
    def __getitem__(self, idx):
        start = idx*self.batch_size
        ed = (idx+1)*self.batch_size
        if ed> self.ImpNum:
            ed = self.ImpNum
            
        clicked_ids = self.clicked_news[start:ed]
        user_title = self.__get_news(clicked_ids)

        click_mask = clicked_ids>0
        click_mask = np.array(click_mask,dtype='float32')

        click_num = click_mask.sum(axis=-1)
        click_num = np.array(click_num,dtype='int32')
        click_num = click_num.reshape((len(click_num),1))

        click_num = torch.LongTensor(click_num).cuda()
        click_mask = torch.FloatTensor(click_mask).cuda()
        user_title = torch.FloatTensor(user_title).cuda()
        
        return [user_title,click_mask,click_num]

In [73]:
def acc(y_true, y_hat):
    y_hat = torch.argmax(y_hat, dim=-1)
    tot = y_true.shape[0]
    hit = torch.sum(y_true == y_hat)
    return hit.data.float() * 1.0 / tot

In [45]:
def evaluate(test_impressions,user_scoring,news_scoring):
    AUC = []
    MRR = []
    nDCG5 = []
    nDCG10 = []
    for i in range(len(test_impressions)):
        labels = test_impressions[i]['labels']
        nids = test_impressions[i]['docs']

        uv = user_scoring[i]

        nvs = news_scoring[nids]
        score = np.dot(nvs,uv)

        auc = roc_auc_score(labels,score)
        mrr = mrr_score(labels,score)
        ndcg5 = ndcg_score(labels,score,k=5)
        ndcg10 = ndcg_score(labels,score,k=10)
    
        AUC.append(auc)
        MRR.append(mrr)
        nDCG5.append(ndcg5)
        nDCG10.append(ndcg10)
        
    AUC = np.array(AUC).mean()
    MRR = np.array(MRR).mean()
    nDCG5 = np.array(nDCG5).mean()
    nDCG10 = np.array(nDCG10).mean()

    
    return AUC, MRR, nDCG5, nDCG10

In [46]:
def get_news_scoring(news_encoder):
    bz = 64
    news_scorings = []
    for i in range(int(np.ceil(len(news_title)/bz))):
        start = bz*i
        ed = bz*(i+1)
        if ed > len(news_title):
            ed = len(news_title)
        data = news_title[start:ed]
        data = torch.LongTensor(data).cuda()
        #print(data.shape)
        ns = news_encoder(data)
        ns = ns.detach().to('cpu').numpy()
        news_scorings.append(ns)
    news_scorings = np.concatenate(news_scorings,axis=0)
    return news_scorings

In [47]:
def get_user_scoring(user_encoder,news_scoring,user_click):
    user_generator = get_hir_user_generator(news_scoring,user_click,32)
    user_scorings = []
    for data in user_generator:
        us = user_encoder(*data)
        us = us.detach().to('cpu').numpy()
        user_scorings.append(us)
    user_scorings = np.concatenate(user_scorings,axis=0)
    return user_scorings

In [48]:
def evaluate_cold(test_impressions,clicks,user_scoring,news_scoring):
    colds = {}
    for i in range(6):
        colds[i] = [[],[],[],[]]
    
    
    for i in range(len(test_impressions)):
        ucs = clicks[i]
        ucs = (ucs>0).sum()
        ucs = int(ucs)
        if ucs>5:
            continue
        
        labels = test_impressions[i]['labels']
        nids = test_impressions[i]['docs']

        uv = user_scoring[i]

        nvs = news_scoring[nids]
        score = np.dot(nvs,uv)

        auc = roc_auc_score(labels,score)
        mrr = mrr_score(labels,score)
        ndcg5 = ndcg_score(labels,score,k=5)
        ndcg10 = ndcg_score(labels,score,k=10)
    
        colds[ucs][0].append(auc)
        colds[ucs][1].append(mrr)
        colds[ucs][2].append(ndcg5)
        colds[ucs][3].append(ndcg10)


    for ucs in range(6):
        colds[ucs] = np.array(colds[ucs]).mean(axis=-1)
    

    
    return colds

In [49]:
def evaluate_cold(test_impressions,clicks,user_scoring,news_scoring):
    AUC = []
    MRR = []
    nDCG5 = []
    nDCG10 = []
    
    
    for i in range(len(test_impressions)):
        ucs = clicks[i]
        ucs = (ucs>0).sum()
        ucs = int(ucs)
        if ucs>=5:
            continue
        if ucs==0:
            continue
        
        labels = test_impressions[i]['labels']
        nids = test_impressions[i]['docs']

        uv = user_scoring[i]

        nvs = news_scoring[nids]
        score = np.dot(nvs,uv)

        auc = roc_auc_score(labels,score)
        mrr = mrr_score(labels,score)
        ndcg5 = ndcg_score(labels,score,k=5)
        ndcg10 = ndcg_score(labels,score,k=10)
    
        AUC.append(auc)
        MRR.append(mrr)
        nDCG5.append(ndcg5)
        nDCG10.append(ndcg10)


    AUC = np.array(AUC).mean()
    MRR = np.array(MRR).mean()
    nDCG5 = np.array(nDCG5).mean()
    nDCG10 = np.array(nDCG10).mean()


    
    return AUC, MRR, nDCG5, nDCG10

In [50]:
def compute_doc_sim(news_scoring):
    z = np.sqrt((news_scoring**2).sum(axis=-1)).reshape((news_scoring.shape[0],1))
    norm_news_scoring = news_scoring/z
    
    random_index = np.random.permutation(len(norm_news_scoring))
    rand_ns = norm_news_scoring[random_index]
    
    scores = (norm_news_scoring*rand_ns).sum(axis=-1).mean()
    
    return scores

In [51]:
def Log(loss,acc,per,K=2):
    NUM = 100//K
    num1 = int(NUM*per)
    num2 = NUM-num1

    print("\r loss=%.3f acc=%.4f  %s%s  %.2f%s" % (loss,acc,'>' * num1,'#'*num2, 100*per,'%'), flush=True, end='')

In [52]:
def model_training_one_epoch(model):
    NUM = train_label.shape[0]//32
    loss = 0.0
    accuary = 0.0
    cnt = 0
    for data in train_generator:
        bz_loss, y_hat = model(*data)
        loss += bz_loss.data.float()
        accuary += acc(data[-1], y_hat)
        optimizer.zero_grad()
        bz_loss.backward()
        optimizer.step()
        cnt += 1
        per = cnt/NUM
        Log(loss/cnt,accuary/cnt,per)
    print()

In [53]:
import numpy as np
import torch
from torch import nn, sub
import torch.nn.functional as F
import os
import math
import torch.optim as optim

In [54]:
class AttentionPooling(nn.Module):
    def __init__(self, d_h, hidden_size, drop_rate):
        super(AttentionPooling, self).__init__()
        self.att_fc1 = nn.Linear(d_h, hidden_size // 2)
        self.att_fc2 = nn.Linear(hidden_size // 2, 1)
        self.drop_layer = nn.Dropout(p=drop_rate)
        torch.nn.init.constant_(self.att_fc2.weight, 0.)


    def forward(self, x, attn_mask=None):

        bz = x.shape[0]
        e = self.att_fc1(x) # (bz, seq_len, 200)
        e = nn.Tanh()(e)
        alpha = self.att_fc2(e) # (bz, seq_len, 1)
        alpha = torch.exp(alpha)

        if attn_mask is not None:
            alpha = alpha * attn_mask.unsqueeze(2)
        alpha = alpha / (torch.sum(alpha, dim=1, keepdim=True) + 1e-8)

        x = torch.bmm(x.permute(0, 2, 1), alpha)
        x = torch.reshape(x, (bz, -1)) # (bz, 400)
        return x

In [55]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super(ScaledDotProductAttention, self).__init__()
        self.d_k = d_k

    def forward(self, Q, K, V, attn_mask=None):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.d_k)
        scores = torch.exp(scores)
        if attn_mask is not None:
            scores = scores * attn_mask
        attn = scores / (torch.sum(scores, dim=-1, keepdim=True)  + 1e-8)
        
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads, d_k, d_v):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model # 300
        self.n_heads = n_heads # 20
        self.d_k = d_k # 20
        self.d_v = d_v # 20
        
        self.W_Q = nn.Linear(d_model, d_k * n_heads) # 300, 400
        self.W_K = nn.Linear(d_model, d_k * n_heads) # 300, 400
        self.W_V = nn.Linear(d_model, d_v * n_heads) # 300, 400
        
        self._initialize_weights()
                
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight, gain=1)
                
    def forward(self, Q, K, V, attn_mask=None):
        residual, batch_size = Q, Q.size(0)
        
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)
        
        if attn_mask is not None:
            attn_mask = attn_mask.unsqueeze(1).expand(batch_size, max_len, max_len) 
            attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1, 1) 
        
        context, attn = ScaledDotProductAttention(self.d_k)(q_s, k_s, v_s, attn_mask) 
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) 
        return context

In [56]:
class NewsEncoder(nn.Module):
    def __init__(self, mode):
        super(NewsEncoder, self).__init__()
        self.mode = mode

        weight = torch.FloatTensor(title_word_embedding_matrix).cuda()
        self.embedding_layer = nn.Embedding.from_pretrained(weight,freeze=False)
        
        if mode == 'NAML':
            self.cnn = nn.Conv1d(300,256,3,padding=1)
        elif mode=='NRMS':
            self.sa = MultiHeadAttention(300,16,16,16)
        self.dropout = nn.Dropout(0.2)

        self.att = AttentionPooling(256,256, 0.2)


    def forward(self, x):
        '''
            x: batch_size, history_len, emb_dim
            xmask: batch_size, history_len
            user_subcate_mask: batch_size, history_len, subcate_len
        '''
        word_emb = self.embedding_layer(x)
        word_emb = self.dropout(word_emb)
        if mode == 'NAML':
            word_emb = word_emb.permute((0,2,1))
            word_vecs = self.cnn(word_emb)
            word_vecs = word_vecs.permute((0,2,1))

        elif mode=='NRMS':
            word_vecs = self.sa(word_emb,word_emb,word_emb)
        news_vec = self.att(word_vecs)

        return news_vec

In [78]:
class UserEncoder(nn.Module):
    def __init__(self,mode, use_mask,use_default,use_num):
        super(UserEncoder, self).__init__()
        
        self.use_mask = use_mask
        self.mode = mode
        self.use_default = use_default
        self.use_num = use_num
        
        if self.mode =='NRMS':
            self.sa = MultiHeadAttention(256,16,16,16)
            
        self.att = AttentionPooling(256,256, 0.2)
        self.dropout = nn.Dropout(0.2)

        self.click_embedding = nn.Embedding(51, 128)
        self.fc = nn.Linear(128, 1)

        self.default_user = nn.Parameter(torch.empty(1, 256).uniform_(-1/np.sqrt(256), 1/np.sqrt(256))).type(torch.FloatTensor)
        #torch.nn.init.constant_(self.default_user, 0.)
        
        
    def forward(self, x, mask,num):
        '''
            x: batch_size, history_len, emb_dim
            xmask: batch_size, history_len
            user_subcate_mask: batch_size, history_len, subcate_len
        '''
        bz = num.shape[0]


        if not self.use_mask:
            mask = None
        
        if self.mode == 'NRMS':
            news_vecs = self.sa(x,x,x,mask)
        else:
            news_vecs = x
            
        news_vecs = self.dropout(news_vecs)
        user_vec = self.att(news_vecs,mask)
        
        if self.use_num:
            click_emb = self.click_embedding(num)
            click_emb = torch.reshape(click_emb,(bz,128))
            click_score = self.fc(click_emb)
            weight = nn.Sigmoid()(click_score)
        
        if self.use_default:
            user_vec = weight*user_vec + (1-weight)*self.default_user.expand_as(user_vec)

            
        return user_vec

In [79]:
class Model(nn.Module):
    def __init__(self,mode,use_mask,use_default,use_num):
        super(Model, self).__init__()

        self.news_encoder = NewsEncoder(mode)
        self.user_encoder = UserEncoder(mode,use_mask,use_default,use_num)
        self.loss_fn = nn.CrossEntropyLoss()
        
    def forward(self,candi_title,click_title,click_mask,click_num,labels):
        # candi_title (bz,1+npratio,30)
        # click_title (bz,50,30)
        # click_mask (bz,50)
        
        bz = candi_title.shape[0]
        candi_title = torch.reshape(candi_title, (bz*(1+npratio),30 ))
        click_title = torch.reshape(click_title, (bz*50,30 ))
        
        candi_vecs = self.news_encoder(candi_title)
        click_vecs = self.news_encoder(click_title)
        
        candi_vecs = torch.reshape(candi_vecs, (bz,1+npratio,256 ))
        click_vecs = torch.reshape(click_vecs, (bz,50,256 ))
        
        user_vec = self.user_encoder(click_vecs,click_mask,click_num)
        
        scores = torch.bmm(candi_vecs, user_vec.unsqueeze(dim=-1)).squeeze(-1)
        loss = self.loss_fn(scores, labels)
        return loss, scores

In [80]:
for v in train_generator:
    break

In [81]:
Res = []

mode = 'NAML'
use_mask = True
use_default = True
use_num = True

model = Model(mode,use_mask,use_default,use_num)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)


for i in range(2):
    model = model.train()
    model_training_one_epoch(model)
    model = model.eval()
    news_encoder = model.news_encoder
    user_encoder = model.user_encoder
    news_scorings = get_news_scoring(news_encoder)
    user_scorings = get_user_scoring(user_encoder,news_scorings,test_user['click'][:200000])
    doc_sim = compute_doc_sim(news_scorings)
    g2 = evaluate(test_impressions[:200000],user_scorings,news_scorings)
    print(g2,doc_sim)

 loss=0.543 acc=0.7217  >>>>>>>>>>>>>>>>>>################################  37.06%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 loss=0.540 acc=0.7239  >>>>>>>>>>>>>>>>>>>>>#############################  42.89%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 loss=0.537 acc=0.7260  >>>>>>>>>>>>>>>>>>>>>>>>##########################  48.73%

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 loss=0.524 acc=0.7363  >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>##  97.69%

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



(0.6725959873401782, 0.32760076857665726, 0.3558208726444875, 0.41249118665038326) 0.14892034


TypeError: evaluate_cold() missing 1 required positional argument: 'news_scoring'